In [73]:
''' Get coin id, name, & symbol'''
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd


url = 'https://api.coingecko.com/api/v3/coins/list'

headers = {
  'accepts': 'application/json',
}

session = Session()
session.headers.update(headers)

parameters = {
    'include_platform':'false'
}
response = session.get(url, params=parameters)
coins = json.loads(response.text)

coins_md_df = pd.DataFrame(coins)
coins_md_df.to_csv("./data/coin_list.csv")
print("Coin meta-data extraction load complete!")


Coin meta-data extraction load complete!


In [6]:
''' convert date to unix timestamp '''
import datetime
import time
 
# assigned regular string date
date_time = datetime.datetime(2022, 6, 1, 0, 0)
 
# print regular python date&time
print("date_time =>",date_time)
 
# displaying unix timestamp after conversion
print("unix_timestamp => ",
      (time.mktime(date_time.timetuple())))

date_time => 2022-06-01 00:00:00
unix_timestamp =>  1654027200.0


In [69]:
''' Generate a list of dates for a range '''
from datetime import date, timedelta
import pandas as pd

start_date = date(2022,4,1)
end_date = date(2022,6,1)
_l_dates = []
while start_date < end_date:
    _l_dates.append(start_date)
    start_date += timedelta(days=1)
print("Generated %d dates ending with %s" % (len(_l_dates),str(end_date)))

Generated 61 dates ending with 2022-06-01


In [72]:
''' Extract selected coin market cap data usng CoinGeko API '''

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

_l_symbols = ["btc", "eth", "xrp", "bch", "ltc", "sol", "ada"]
_str_coin_id = "bitcoin"
url = "https://api.coingecko.com/api/v3/coins/"+_str_coin_id+"/history"

headers = {
  'Accepts': 'application/json',
}

session = Session()
session.headers.update(headers)
_l_mdate = []
_l_symbol = []
_l_mcap = []
try:
    print("Extracting market cap data for %s" % _str_coin_id)
    for date in _l_dates:
        parameters = {
            'date': date.strftime("%d-%m-%Y"),
            'localization':'false',
        }
        data = []
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        _l_mdate.append(date)
        _l_symbol.append(data["symbol"])
        if "market_data" in data.keys():    
            _l_mcap.append(data["market_data"]["market_cap"]["usd"])
        else:
            _l_mcap.append(0)
    market_df = pd.DataFrame({"Date" : _l_mdate,
                              "Symbol" : _l_symbol,
                              "market_cap" : _l_mcap
                             })
    
    if market_df.shape[0] > 0:
        _file_name = "./data/market_cap_"+_str_coin_id+".csv"
        print("save data to ", _file_name)
        market_df.to_csv(_file_name, sep=",")
    print("Data load complete!")

except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)

Extracting market cap data for bitcoin
save data to  ./data/market_cap_bitcoin.csv
Data load complete!
